In [1]:

import sys
sys.path.append('/home/techt/Desktop/a4s')


### Step 1: Initial Data Loading

In [2]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Polycystic_Ovary_Syndrome/GSE43322'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Subcutaneous adipose tissue gene expression in PCOS"
!Series_summary	"This SuperSeries is composed of the SubSeries listed below."
!Series_overall_design	"Refer to individual Series"
Sample Characteristics Dictionary:
{0: ['gender: Female'], 1: ['age (yrs): 39', 'age (yrs): 32', 'age (yrs): 22', 'age (yrs): 25', 'age (yrs): 26', 'age (yrs): 28', 'age (yrs): 27', 'age (yrs): 36', 'age (yrs): 37', 'age (yrs): 34', 'age (yrs): 30', 'age (yrs): 40', 'age: 39', 'age: 32', 'age: 22', 'age: 25', 'age: 26', 'age: 28', 'age: 27'], 2: ['bmi: 38.24', 'bmi: 37.42', 'bmi: 46.8', 'bmi: 36.88', 'bmi: 29.55', 'bmi: 31.64', 'bmi: 46.22', 'bmi: 38.37', 'bmi: 34.9', 'bmi: 34.56', 'bmi: 47.4', 'bmi: 36.4', 'bmi: 29.4', 'bmi: 47.8', 'bmi: 37.3'], 3: ['condition: polycystic ovary syndrome (PCOS)', 'condition: control'], 4: ['tissue: subcutaneous adipose tissue'], 5: [nan, 'agent: placebo', 'agent: LC n-3 PUFA']}


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
is_gene_available = False
trait_row = age_row = gender_row = None  # set to different values when applicable
convert_trait = convert_age = convert_gender = None  # define the functions when applicable

# Check if gene expression data is available based on background information.
series_title = "!Series_title	\"Subcutaneous adipose tissue gene expression in PCOS\""
if "gene expression" in series_title.lower():
    is_gene_available = True

# Analyze the Sample Characteristics Dictionary for variable availability.
sample_characteristics = {
    0: ['gender: Female'], 
    1: ['age (yrs): 39', 'age (yrs): 32', 'age (yrs): 22', 'age (yrs): 25', 'age (yrs): 26', 'age (yrs): 28', 'age (yrs): 27', 'age (yrs): 36', 'age (yrs): 37', 'age (yrs): 34', 'age (yrs): 30', 'age (yrs): 40', 'age: 39', 'age: 32', 'age: 22', 'age: 25', 'age: 26', 'age: 28', 'age: 27'], 
    2: ['bmi: 38.24', 'bmi: 37.42', 'bmi: 46.8', 'bmi: 36.88', 'bmi: 29.55', 'bmi: 31.64', 'bmi: 46.22', 'bmi: 38.37', 'bmi: 34.9', 'bmi: 34.56', 'bmi: 47.4', 'bmi: 36.4', 'bmi: 29.4', 'bmi: 47.8', 'bmi: 37.3'], 
    3: ['condition: polycystic ovary syndrome (PCOS)', 'condition: control'], 
    4: ['tissue: subcutaneous adipose tissue'], 
    5: [float('nan'), 'agent: placebo', 'agent: LC n-3 PUFA']
}

# Identify keys for 'Polycystic_Ovary_Syndrome', 'age', and 'gender'
trait_row = 3 if len(set([v.split(':')[1].strip().lower() for v in sample_characteristics[3]])) > 1 else None
age_row = 1 if len(set([v.split(':')[1].strip() for v in sample_characteristics[1]])) > 1 else None
gender_row = 0 if len(set([v.split(':')[1].strip().lower() for v in sample_characteristics[0]])) > 1 else None

# Define conversion functions
def convert_trait(value):
    try:
        _, val = value.split(':')
        val = val.strip().lower()
        return 1 if 'polycystic' in val else 0 if 'control' in val else None
    except:
        return None

def convert_age(value):
    try:
        _, val = value.split(':')
        return float(val.strip())
    except:
        return None

def convert_gender(value):
    try:
        _, val = value.split(':')
        val = val.strip().lower()
        return 1 if 'male' in val else 0 if 'female' in val else None
    except:
        return None

# Save cohort information
save_cohort_info('GSE43322', './preprocessed/Polycystic_Ovary_Syndrome/cohort_info.json', is_gene_available, trait_row is not None)

# If clinical data is available, extract clinical features
if trait_row is not None:
    selected_clinical_data = geo_select_clinical_features(clinical_data, 'Polycystic_Ovary_Syndrome', trait_row, convert_trait, age_row, convert_age, gender_row, convert_gender)
    csv_path = './preprocessed/Polycystic_Ovary_Syndrome/trait_data/GSE43322.csv'
    selected_clinical_data.to_csv(csv_path)
    print(preview_df(selected_clinical_data))


{'GSM1059640': [1.0, 39.0], 'GSM1059641': [1.0, 32.0], 'GSM1059642': [1.0, 22.0], 'GSM1059643': [1.0, 25.0], 'GSM1059644': [1.0, 26.0], 'GSM1059645': [1.0, 28.0], 'GSM1059646': [1.0, 27.0], 'GSM1059647': [1.0, 28.0], 'GSM1059648': [0.0, 36.0], 'GSM1059649': [0.0, 37.0], 'GSM1059650': [0.0, 39.0], 'GSM1059651': [0.0, 34.0], 'GSM1059652': [0.0, 30.0], 'GSM1059653': [0.0, 22.0], 'GSM1059654': [0.0, 40.0], 'GSM1059686': [1.0, 39.0], 'GSM1059687': [1.0, 39.0], 'GSM1059688': [1.0, 32.0], 'GSM1059689': [1.0, 32.0], 'GSM1059690': [1.0, 22.0], 'GSM1059691': [1.0, 22.0], 'GSM1059692': [1.0, 25.0], 'GSM1059693': [1.0, 25.0], 'GSM1059694': [1.0, 26.0], 'GSM1059695': [1.0, 26.0], 'GSM1059696': [1.0, 28.0], 'GSM1059697': [1.0, 28.0], 'GSM1059698': [1.0, 27.0], 'GSM1059699': [1.0, 27.0], 'GSM1059700': [1.0, 28.0], 'GSM1059701': [1.0, 28.0]}


### Step 3: Gene Data Extraction

In [4]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row ids for the following step.
print(gene_data.index[:20])


Index(['100009676_at', '10001_at', '10002_at', '10003_at', '100048912_at',
       '100049587_at', '100049716_at', '10004_at', '10005_at', '10006_at',
       '10007_at', '10008_at', '10009_at', '1000_at', '100101467_at',
       '10010_at', '10011_at', '100127206_at', '100127888_at', '100127889_at'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
requires_gene_mapping = True


### Step 5: Gene Annotation (Conditional)

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the soft file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['1_at', '10_at', '100_at', '1000_at', '100009676_at'], 'ORF': ['A1BG', 'NAT2', 'ADA', 'CDH2', 'LOC100009676'], 'ENTREZ_GENE_ID': [1.0, 10.0, 100.0, 1000.0, 100009676.0], 'Description': ['alpha-1-B glycoprotein', 'N-acetyltransferase 2 (arylamine N-acetyltransferase)', 'adenosine deaminase', 'cadherin 2, type 1, N-cadherin (neuronal)', 'hypothetical LOC100009676'], 'SPOT_ID': [nan, nan, nan, nan, nan]}


### Step 6: Gene Identifier Mapping

In [7]:
# 1. Identify the keys for gene mapping.
identifier_key = 'ID'
gene_symbol_key = 'ORF'

# 2. Get the dataframe storing the mapping between probe IDs and genes using the 'get_gene_mapping' function from the library.
mapping_df = get_gene_mapping(gene_annotation, identifier_key, gene_symbol_key)

# 3. Apply the mapping with the 'apply_gene_mapping' function from the library, and name the resulting gene expression dataframe "gene_data".
gene_data = apply_gene_mapping(gene_data, mapping_df)
